<a href="https://colab.research.google.com/github/DLSeed/DeepLearning/blob/main/sovits_f0%E4%B8%80%E9%94%AE%E8%AE%AD%E7%BB%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[sovits合集导航](https://github.com/IceKyrin/sovits_guide)**

**本colab为Rcell版sovits2.0(f0分支)，模型仅在三个一键脚本内部互通**

[sovits_f0](https://github.com/innnky/so-vits-svc)

**默认每隔2000次step保存一次，可在“每隔多少次step保存一次断点”部分进行修改。**

**在看到进度save之前不要轻易退出，以免丢失进度。**

在此基础上修改为sovits配置，配合sovits一键制作数据集使用
**为达最佳效果，建议下载公开歌声数据集opencpop，进行多人模型训练**

**格式参考vits专栏三件套（评论区）**[vits注解](https://www.bilibili.com/read/cv18478187)

95%的问题都可以参考专栏解决，剩下的我也不会了

前置：[一键制作数据集](https://colab.research.google.com/drive/1avWZ_N5BsQcq45XkwQkDpmp912CLZS0n#scrollTo=xx2oAf90btEy)

后置：[一键合成](https://colab.research.google.com/drive/1F3VpHCi9eridGw1F1hbqR7qhXGKuSCus#scrollTo=vjkgBV7j2cVJ)




In [ ]:
# 查看显卡
!nvidia-smi

In [ ]:
#@title 准备
#@markdown 定义工具函数 `run_command` `run_command_by_line` `get_symbols` 和 `get_tensorboard_showing`
# forked from https://www.endpointdev.com/blog/2015/01/getting-realtime-output-using-python/
import os
import subprocess
def run_command(command_args):
    def print_pipe(raw):
        return print(raw.decode("utf-8"), end='')
    try:
      process = subprocess.Popen(command_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
      out, err = process.communicate()
    except:
      pass
    print_pipe(out)
    print_pipe(err)
    rc = process.poll()
    return rc

def run_command_by_line(command_args):
    def print_pipe(raw):
        return print(raw.decode("utf-8"), end='')
    with subprocess.Popen(command_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE) as process:
      while process.poll() is None:
        print_pipe(process.stdout.readline())
      [print_pipe(line) for line in process.stderr.readlines()]
    return

def get_tensorboard_showing(logdir):
    from multiprocessing import Process
    from tensorboard import notebook
    import tensorflow as tf
    import time

    def run_tb():
        run_command_by_line(["tensorboard","--reload_interval", "30",  "--logdir", logdir, "--bind_all"])
    
    def monitor_tb():
        while True:
            try:
                notebook.display(height=998)
                break
            except Exception as e:
                print(e)
                time.sleep(3)

    if param_enable_tb:
        Process(target=run_tb).start()
        Process(target=monitor_tb).start()

In [ ]:
#@title 下载依赖库
#@markdown 取消勾选则不会节省空间
colab_save_space = True #@param {type:"boolean"}
os.chdir('/content')
run_command_by_line(["git", "clone", "https://github.com/IceKyrin/sovits_f0_train.git", "-b", "main" if colab_save_space else "main"])
os.chdir('/content/sovits_f0_train')
!pip install -r requirements.txt
!sudo apt-get install espeak -y
!sudo apt-get install p7zip-full p7zip-rar
!pip install demjson

In [ ]:
#@title 加载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 解压数据集
#@markdown 压缩包名称(不带zip)
DATASETNAME = "out_qiu"  #@param {type:"string"}
#@markdown 压缩包路径(这个名字是接着上一篇的输出zip)
ZIP_PATH = "/content/drive/MyDrive/dataset/out_qiu.zip"  #@param {type:"string"}
DATASETPATH = "/content/" + DATASETNAME
!cp {ZIP_PATH} {DATASETPATH}.zip
%cd /content
!unzip /content/{DATASETNAME}.zip

In [ ]:
#@title 生成配置文件 在configs下，记得自己下载保留一份，下次用
# forked from https://github.com/CjangCjengh/vits/blob/main/configs/japanese_ss_base2.json
#@markdown 配置文件名称
json_filename = "qiu.json" #@param {type:"string"}
#@markdown 训练次数
hparams_epochs = 3000 #@param {type:"integer"}
#@markdown 每隔多少次step保存一次断点
hparams_eval_interval = 2000 #@param {type:"integer"}
#@markdown 单次step的文件数（建议在16以内）
hparams_batch_size = 16 #@param {type:"integer"}
#@markdown 训练集文件列表
hparams_training_files = "/content/qiu/train.txt" #@param {type:"string"}
#@markdown 验证集文件列表
hparams_validation_files = "/content/qiu/val.txt"#@param {type:"string"}
#@markdown 人物名，多个人物用英文逗号隔开
hparams_speaker = "maolei,wu,opencpop,opera,x,y,z" #@param {type:"string"}
#@markdown 模型名
hparams_model_name = "qiu" #@param {type:"string"}

speakers = [speaker.strip() for speaker in hparams_speaker.split(",")]
print("speakers: ")
for i, speaker in enumerate(speakers):
  print("\t{a}: {b}".format(a=i, b=speaker))
training_json = {
  "train": {
    "log_interval": 200,
    "eval_interval": hparams_eval_interval,
    "seed": 1234 ,
    "epochs": hparams_epochs,
    "learning_rate": 2e-4,
    "betas": [0.8, 0.99],
    "eps": 1e-9,
    "batch_size": hparams_batch_size,
    "fp16_run": True,
    "lr_decay": 0.999875,
    "segment_size": 8192,
    "init_lr_ratio": 1,
    "warmup_epochs": 0,
    "c_mel": 45,
    "c_kl": 1.0
  },
  "data": {
    "training_files": hparams_training_files,
    "validation_files": hparams_validation_files,
    "text_cleaners":["english_cleaners2"],
    "max_wav_value": 32768.0,
    "sampling_rate": 22050,
    "filter_length": 1024,
    "hop_length": 256,
    "win_length": 1024,
    "n_mel_channels": 80,
    "mel_fmin": 0.0,
    "mel_fmax": None,
    "add_blank": True,
    "n_speakers": len(speakers) if len(speakers) > 1 else 0
  },
  "model": {
    "inter_channels": 192,
    "hidden_channels": 256,
    "filter_channels": 768,
    "n_heads": 2,
    "n_layers": 6,
    "kernel_size": 3,
    "p_dropout": 0.1,
    "resblock": "1",
    "resblock_kernel_sizes": [3,7,11],
    "resblock_dilation_sizes": [[1,3,5], [1,3,5], [1,3,5]],
    "upsample_rates": [8,8,2,2],
    "upsample_initial_channel": 512,
    "upsample_kernel_sizes": [16,16,4,4],
    "n_layers_q": 3,
    "use_spectral_norm": False
  },
  "speakers": speakers
}

if len(speakers) > 1:
  training_json["model"]["gin_channels"] = 256

import demjson
os.chdir('/content/sovits_f0_train/configs')
training_json_text = demjson.encode(training_json)
with open(json_filename, "w") as file:
  file.write(training_json_text)

os.chdir('/content/sovits_f0_train')



In [ ]:
#@title 预处理
os.chdir('/content/sovits_f0_train/monotonic_align')
!python setup.py build_ext --inplace
os.chdir('/content/sovits_f0_train')

若使用预模型（**现在没有**），直接在预模型基础上训练，节省时间

下载后需要先点下一步训练，待自动在自己的谷歌盘生成模型文件夹后；

把这两个文件从content移动到自己的谷歌盘相应位置、替换自动生成的G_0、D_0；再次训练，会自动读取新的pth

使用预模型读取pth文件后，文件名的G_steps.pth，steps会乱是正常现象（使用v100训练，batch_size为32，colab显存不够改成了12，所以会乱掉，保存最新生成的模型即可）

In [ ]:
#@markdown 是否使用预模型（**现在没有**）
pre_pth = False #@param {type:"boolean"}
if pre_pth:
  !gdown --id '1s7zrvJzGqGB6YGnD2t6UppYy6DHZLO7E' --output /content/D_422000.pth
  !gdown --id '1RcXzNNJ8XxtE2kNdY8FZfBoPqYTJ1hl_' --output /content/G_422000.pth
  !gdown --id '12pClKJwraPeIRc1mvOlSeax5QGcD6_6_' --output /content/sovits_f0_train/configs/lang_pre.json

In [ ]:
#@title 训练

#@markdown 启用tensorboard可视化数据，运行两三分钟不显示就刷新一下；

#@markdown epoch次数看/content/drive/MyDrive/数据集名/train.log

#@markdown 数据集名
dataset_name = "qiu"  # @param {type:"string"}
#@markdown 模型自动保存到 /云盘路径/模型名 ,记得定期清理空间！
param_enable_tb = True  # @param {type:"boolean"}
if param_enable_tb:
  #@markdown 云盘路径（一般不改） （定时清云盘！！！pth里保留最新的俩，其他的都别删）
  logdir = "/content/drive/MyDrive/"  # @param {type:"string"}
  new_pth_dir = os.path.join(logdir, dataset_name)
  get_tensorboard_showing(new_pth_dir)
os.chdir('/content/sovits_f0_train')
#@markdown 配置文件json名（使用预模型则为lang_pre.json）
json_filename = "qiu.json"  # @param {type:"string"}
# 这里魔改过until.py的args
run_command_by_line(["python", "train_ms.py" if len(speakers) > 1 else "train.py", "-c", "configs/{json}".format(json=json_filename), "-m", hparams_model_name, "-l",logdir])

In [ ]:
#@title 精简模型
import torch
#@markdown 模型名，数字是最新的模型steps
new_model_name = "G_207000.pth"  # @param {type:"string"}
checkpoint_dict = torch.load(f"{new_pth_dir}/{new_model_name}")
iteration = checkpoint_dict['iteration']
learning_rate = checkpoint_dict['learning_rate']
optimizer = checkpoint_dict['optimizer']
saved_state_dict = checkpoint_dict['model']
print(iteration)
#@markdown 输出xxx_epoch.pth
torch.save({'model': saved_state_dict,
  'iteration': None,
  'optimizer': None,
  'learning_rate': None}, f'{new_pth_dir}/{iteration}_epochs.pth')